In [3]:
import numpy as np
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [4]:
batch_size = 64
transformations = transform=transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize((200,200), antialias=True)])

trainset = datasets.ImageFolder('/dtu/datasets1/02514/hotdog_nothotdog/train', transform=transformations)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)

testset = datasets.ImageFolder('/dtu/datasets1/02514/hotdog_nothotdog/test', transform=transformations)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=1)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.convolutional = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding='same'),
                nn.LeakyReLU(),
                nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding='same'),
                nn.LeakyReLU(),
                nn.MaxPool2d(kernel_size=2),
                nn.Dropout(0.3),
                nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding='same'),
                nn.LeakyReLU(),
                nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding='same'),
                nn.LeakyReLU(),
                nn.MaxPool2d(kernel_size=2),
                nn.Dropout(0.3),
                nn.BatchNorm2d(64)
                )
    
        self.fully_connected = nn.Sequential(
                nn.Linear(50*50*64, 500),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(500, 10))
#                 nn.Softmax(dim=1))    
    
    def forward(self, x):
        x = self.convolutional(x)
        #reshape x so it becomes flat, except for the first dimension (which is the minibatch)
        x = x.view(x.size(0), -1)
        x = self.fully_connected(x)
        return x

In [7]:
def train(model, optimizer, num_epochs=10):
    
    def loss_fun(output, target):
        return F.cross_entropy(output, target)
    
    out_dict = {'train_acc': [],
              'test_acc': [],
              'train_loss': [],
              'test_loss': []}
  
    for epoch in tqdm(range(num_epochs), unit='epoch'):
        model.train()
        #For each epoch
        train_correct = 0
        train_loss = []
        for minibatch_no, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
            data, target = data.to(device), target.to(device)
            #Zero the gradients computed for each weight
            optimizer.zero_grad()
            #Forward pass your image through the network
            output = model(data)
            #Compute the loss
            loss = loss_fun(output, target)
            #Backward pass through the network
            loss.backward()
            #Update the weights
            optimizer.step()

            train_loss.append(loss.item())
            #Compute how many were correctly classified
            predicted = output.argmax(1)
            train_correct += (target==predicted).sum().cpu().item()
            
        #Comput the test accuracy
        test_loss = []
        test_correct = 0
        model.eval()
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            with torch.no_grad():
                output = model(data)
            test_loss.append(loss_fun(output, target).cpu().item())
            predicted = output.argmax(1)
            test_correct += (target==predicted).sum().cpu().item()
            
        out_dict['train_acc'].append(train_correct/len(trainset))
        out_dict['test_acc'].append(test_correct/len(testset))
        out_dict['train_loss'].append(np.mean(train_loss))
        out_dict['test_loss'].append(np.mean(test_loss))
        
        print(f"Loss train: {np.mean(train_loss):.3f}\t test: {np.mean(test_loss):.3f}\t",
              f"Accuracy train: {out_dict['train_acc'][-1]*100:.1f}%\t test: {out_dict['test_acc'][-1]*100:.1f}%")
        
    return out_dict

In [8]:
model = Network()
model.to(device)
out_dict_sgd = train(model, torch.optim.Adam(model.parameters(), lr=0.001))

  0%|          | 0/10 [00:00<?, ?epoch/s]

  0%|          | 0/32 [00:00<?, ?it/s]

Loss train: 1.056	 test: 1.931	 Accuracy train: 60.3%	 test: 60.4%


  0%|          | 0/32 [00:00<?, ?it/s]

Loss train: 0.560	 test: 0.506	 Accuracy train: 74.2%	 test: 75.7%


  0%|          | 0/32 [00:00<?, ?it/s]

Loss train: 0.520	 test: 0.944	 Accuracy train: 74.0%	 test: 74.1%


  0%|          | 0/32 [00:00<?, ?it/s]


KeyboardInterrupt

